# Final Tutorial Notebook
#### Emily Gong, Robert Morrison

In this notebook we will explore the transcripts from UN General Debates. Our goal is to understand what differences exist between speakers of different language backgrounds

In [14]:
import numpy as np
import pandas as pd

import requests
import bs4

In [4]:
data = pd.read_csv("un-general-debates.csv")
data.head()

,session,year,country,text
0,44,1989,MDV,﻿It is indeed a pleasure for me and the member...
1,44,1989,FIN,"﻿\nMay I begin by congratulating you. Sir, on ..."
2,44,1989,NER,"﻿\nMr. President, it is a particular pleasure ..."
3,44,1989,URY,﻿\nDuring the debate at the fortieth session o...
4,44,1989,ZWE,﻿I should like at the outset to express my del...


In [8]:
print(len(data))

7507


In [6]:
countries = data["country"].unique()
print(countries)
print(len(countries))

['MDV' 'FIN' 'NER' 'URY' 'ZWE' 'PHL' 'SDN' 'RUS' 'CHN' 'ESP' 'SUR' 'ARG'
 'SLV' 'MYS' 'NPL' 'PRT' 'COL' 'BLR' 'MAR' 'LCA' 'EGY' 'MEX' 'BEL' 'BRN'
 'RWA' 'CAN' 'ALB' 'GRC' 'KNA' 'GUY' 'LBR' 'ATG' 'MOZ' 'JPN' 'YDYE' 'GAB'
 'BGD' 'SWE' 'TUR' 'TCD' 'SYR' 'CMR' 'JAM' 'LUX' 'ITA' 'AGO' 'CRI' 'CSK'
 'BFA' 'MNG' 'BHR' 'HTI' 'OMN' 'CIV' 'TGO' 'CYP' 'MUS' 'MMR' 'ARE' 'GTM'
 'GRD' 'LBY' 'LKA' 'TZA' 'SGP' 'NOR' 'LAO' 'ISL' 'AFG' 'CHL' 'DMA' 'UKR'
 'KEN' 'BLZ' 'FRA' 'MLI' 'VCT' 'VEN' 'MLT' 'GHA' 'GIN' 'GBR' 'ISR' 'YUG'
 'BRB' 'IRQ' 'HUN' 'AUT' 'POL' 'GNB' 'BWA' 'MRT' 'SWZ' 'DNK' 'DOM' 'MDG'
 'NIC' 'BDI' 'CUB' 'IRN' 'PAK' 'SEN' 'BGR' 'YEM' 'STP' 'NLD' 'VUT' 'BOL'
 'PNG' 'SLB' 'DEU' 'ROU' 'KHM' 'TUN' 'BRA' 'IND' 'IDN' 'AUS' 'COD' 'HND'
 'GNQ' 'FJI' 'IRL' 'DZA' 'USA' 'LSO' 'GMB' 'PER' 'DDR' 'THA' 'JOR' 'COG'
 'NGA' 'ECU' 'SAU' 'QAT' 'SYC' 'ETH' 'TTO' 'PRY' 'VNM' 'NZL' 'PAN' 'MWI'
 'DJI' 'BEN' 'SOM' 'ZMB' 'CPV' 'BHS' 'KWT' 'UGA' 'COM' 'ZAF' 'LBN' 'SLE'
 'KOR' 'BIH' 'TON' 'EU' 'HRV' 'NRU' 'TUV' 'NAM' 'S

In [7]:
years = data["year"].unique()
print(years)
print(len(years))
# 46 Examples of each country

[1989 1970 2013 1985 2008 1991 1986 2002 1975 1996 2012 1997 1978 1988
 2010 1984 1995 2009 1971 1976 1983 1979 1999 2005 1987 1982 1998 2003
 2004 1980 2014 2011 1974 2015 1993 1977 1981 2000 1992 1990 1973 1994
 1972 2006 2007 2001]
46


In [44]:
# Getting a decoding table and access to each countries Wikipedia page
url = "https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3"
r = requests.get(url)
soup = bs4.BeautifulSoup(r.text)

In [56]:
found = soup.find("div", class_="plainlist").find("ul")

In [77]:
lookup = {} # dict {country code : [Country Name, wikipedia link]}

for child in found.children:
    if isinstance(child, bs4.Tag):
        c_code = child.find("span").text
        link = child.find("a").get("href")
        country = child.find("a").text
        
        lookup[c_code] = [country, link]

In [78]:
data["name"] = [lookup[code] for code in data["country"]]

KeyError: 'YDYE'

In [84]:
data[data["country"] == "YEM"]

,session,year,country,text
103,44,1989,YEM,﻿Allow me at the outset of my statement to exp...
265,68,2013,YEM,"Allow \nme, at the outset to extend sincere my..."
541,40,1985,YEM,"Mr. President, allow me at the outset to expre..."
736,63,2008,YEM,I should \nlike to start by congratulating Mr....
878,46,1991,YEM,﻿It gives me pleasure to congratulate the Pres...
936,41,1986,YEM,As I stand before the General Assembly today ...
1208,57,2002,YEM,"﻿At the\noutset, I would like to express to yo..."
1282,30,1975,YEM,\n109.\t I should like to extend to Mr. Gaston...
1534,51,1996,YEM,﻿It is a pleasure for me to warmly congratulat...
1575,67,2012,YEM,"﻿To begin\nwith, I offer my deep congratulatio..."


In [85]:
data[data["country"] == "YDYE"]

'''
also testing merge
'''

,session,year,country,text
34,44,1989,YDYE,﻿\nThe election of Ambassador Garba of Nigeria...
523,40,1985,YDYE,It gives me great pleasure to extend to Mr. Ja...
986,41,1986,YDYE,We should like to express our warmest heartfel...
1344,30,1975,YDYE,"117.\t It gives me pleasure to express to you,..."
1966,33,1978,YDYE,"﻿53.\tMr. President, it gives me great pleasur..."
2211,43,1988,YDYE,﻿It is a pleasure for me to participate in a s...
2415,39,1984,YDYE,﻿It gives me great pleasure to congratulate Mr...
2935,26,1971,YDYE,"31.\t Mr. President, may I congratulate you on..."
3038,31,1976,YDYE,"Mr. President, it gives me great pleasure to c..."
3211,38,1983,YDYE,"﻿94.\t Mr. President, it gives me great pleasu..."
